In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('D:\\Dropbox\\stempy')
sys.path.append('D:\\Dropbox\\stemplot')
    
from stemplot import *

from stempy.io import *
from stempy.clustering import *
from stempy.plot import *
from stempy.denoise import *
from stempy.utils import *
from stempy.feature import *
from stempy.spatial import *
from stempy.manifold import *

In [2]:
dp = r'D:\Dropbox\data\Leyi\Dopant Valency Project\TiWSe2_total 12 datsets\TiWSe2\TiWSe2_20211028'+'\\'

num = 2

file_name1 = dp + 'ADF2_{}.dm4'.format(num)

# load image
img = load_image(file_name1)

img = normalize_image(img, 0, 1)

In [3]:
img1 = remove_bg(img, 5)

imshow(img1)

In [4]:
get_window_size(img)

8

## denoising

In [6]:
n_components = 32
patch_size = 32
extraction_step = 2
imgf = denoise_svd(img1, n_components, patch_size, extraction_step) 

Extracting reference patches...
done in 0.19s.
Singular value decomposition...
done in 1.89s.
Reconstructing...
done in 1.04s.


In [7]:
imshow(imgf)

In [8]:
threshold = 0.1
pts = local_max(imgf, min_distance=1, threshold=threshold)

In [9]:
size = 12
kp = KeyPoints(pts, img1, size)
ps = kp.extract_patches(size) 

In [10]:
imshow(ps)

In [11]:
zps = ZPs(n_max=6, size=ps.shape[1])
zps.fit(ps)
X = zps.moments
X_rot = X.rotinv() 

In [12]:
plot_pca(X, 2)

In [13]:
fg = ForceGraph8(X = X,
n_neighbors=10,
init_mode = 'pca',
num_iterations = 100,
num_negative_samples=5,
force_params1=(0, 2, 1, 1),
force_params2=(2, 5, 5, 1),
divide=0.5)

from time import time
t0 = time()
xy = fg.fit_transform(X)

Construct graph from data...
Initialize 2-d embedding using PCA layout...


In [14]:
l= interactive_clusters(xy, img, kp.pts, ps, clip=False) 

In [15]:
l.ind

array([  24,   95,  133,  157,  194,  250,  310,  447,  597,  612,  626,
        671,  690,  851,  877,  882,  913, 1041, 1109, 1112, 1120, 1351,
       1365, 1407, 1437, 1470, 1534, 1539, 1697, 1716, 1793, 1822, 2018,
       2064, 2167, 2213, 2386, 2407, 2416, 2445, 2499, 2560, 2719, 2829,
       2915, 2920, 2930, 2942, 2944, 2957, 3025, 3041, 3100, 3206, 3229,
       3243, 3359, 3367, 3375, 3393, 3523, 3531, 3562, 3594, 3676, 3703,
       3892, 4037, 4263, 4378, 4391, 4440, 4449, 4478, 4485, 4576, 4586,
       4590, 4642, 4665, 4727, 4740, 4761, 4772, 4809, 4854, 4863, 4897,
       4910, 4925, 4975, 4982, 5030, 5055, 5077, 5398, 5413, 5415, 5580,
       5628, 5690, 5699, 5882, 5883, 5891, 5970, 5972, 6031, 6111, 6154,
       6167, 6248, 6264, 6349, 6387, 6428, 6432, 6438, 6541, 6554, 6558,
       6562, 6565, 6651, 6674, 6693, 6934, 7006, 7061, 7142, 7144, 7194,
       7334, 7348, 7355, 7421, 7452, 7603, 7682, 7697, 7745, 7846],
      dtype=int64)

In [16]:
kp.pts

array([[ 48,   8],
       [ 78,   8],
       [121,   8],
       ...,
       [452, 504],
       [461, 504],
       [482, 504]], dtype=int64)

In [17]:
np.save('pts_Ti_25.npy', kp.pts)

In [18]:
np.save('ind_Ti_25.npy', l.ind)